In [235]:
import tbapy
import json,csv
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat
import scipy

import csv
tba = tbapy.TBA('OXCHGYRjLglaSVGWzbxHlvAkoFP5IN8Ja0yPXiV3aN6XP9zZhOgm9FJsPxao7Q71')

event = "2020mndu"
df = pd.DataFrame(tba.event_matches(event))

df established, clean up

In [236]:
list(df.columns)

['actual_time',
 'alliances',
 'comp_level',
 'event_key',
 'key',
 'match_number',
 'post_result_time',
 'predicted_time',
 'score_breakdown',
 'set_number',
 'time',
 'videos',
 'winning_alliance']

We want to remove all pointless information from our dataframe

In [237]:
df.drop(columns=['actual_time','event_key','post_result_time','predicted_time','set_number','time','videos'])
#df.to_csv("EventData.csv")
print(df.columns)

Index(['actual_time', 'alliances', 'comp_level', 'event_key', 'key',
       'match_number', 'post_result_time', 'predicted_time', 'score_breakdown',
       'set_number', 'time', 'videos', 'winning_alliance'],
      dtype='object')


Getting a bunch of dictionary values inside of scorebreakdown, lets see how we can convert that to columns

In [238]:
ad = df.loc[0,'alliances']
print(ad)
ad['red']['team_keys'][2][3:]

{'blue': {'dq_team_keys': [], 'score': 96, 'surrogate_team_keys': [], 'team_keys': ['frc4728', 'frc5348', 'frc2526']}, 'red': {'dq_team_keys': [], 'score': 150, 'surrogate_team_keys': [], 'team_keys': ['frc167', 'frc1816', 'frc5464']}}


'5464'

In [239]:
#score dataframe

score = df.loc[0,'score_breakdown']

data = []

matchCount = len(df.loc[:,'match_number'])

for i in range(matchCount):
    iScore = df.loc[i,'score_breakdown']
    iAlliance = df.loc[i,'alliances']

    bScore = iScore['blue']
    bRobots = iAlliance['blue']['team_keys']

    rScore = iScore['red']
    rRobots = iAlliance['red']['team_keys']

        #frcXXXX -> XXXX
    #Append alliance numbers onto rScore Dictionary
    bScore['alliance1'] = bRobots[0][3:]
    bScore['alliance2'] = bRobots[1][3:]
    bScore['alliance3'] = bRobots[2][3:]
    bScore['matchKey'] = df.loc[i,'key']

    rScore['alliance1'] = rRobots[0][3:]
    rScore['alliance2'] = rRobots[1][3:]
    rScore['alliance3'] = rRobots[2][3:]
    rScore['matchKey'] = df.loc[i,'key']

    data.append(bScore)
    data.append(rScore)

sdf = pd.DataFrame(data)
print(sdf)
#hell yea

     adjustPoints  autoCellPoints  autoCellsBottom  autoCellsInner  \
0               0              30                1               0   
1               0              44                0               0   
2               0              30                3               0   
3               0              46                0               1   
4               0              22                0               1   
..            ...             ...              ...             ...   
215             0              22                0               1   
216             0              32                1               1   
217             0              32                0               0   
218             0              26                0               1   
219             0              20                0               0   

     autoCellsOuter  autoInitLinePoints  autoPoints  controlPanelPoints  \
0                 7                  15          45                   0   
1        

In [240]:
sdf.to_csv("Refined_Event_Data.csv")

In [241]:
print(sdf.columns)

#Lets sort each columns into a group by using a list for reindexing the columns.
#I just seperated these by line but in the dataframe the categories will be adjacent
reorderColumnList = [ 'matchKey','alliance1','alliance2','alliance3',

    'autoCellPoints','autoCellsBottom','autoCellsInner', 'autoCellsOuter', 'autoInitLinePoints', 'autoPoints', 'initLineRobot1', 'initLineRobot2', 'initLineRobot3',
                                                                                                                                                                                    'teleopCellPoints','teleopCellsBottom','teleopCellsInner','teleopCellsOuter','teleopPoints',

                     'endgamePoints', 'endgameRobot1', 'endgameRobot2' , 'endgameRobot3', 'endgameRungIsLevel','tba_numRobotsHanging',

                      'totalPoints',

                      'rp','shieldEnergizedRankingPoint','shieldOperationalRankingPoint','tba_shieldEnergizedRankingPointFromFoul','stage1Activated','stage2Activated','stage3Activated','stage3TargetColor','foulCount','techFoulCount', 'foulPoints',

                     'adjustPoints', 'controlPanelPoints'
]
sdf = sdf.reindex(columns = reorderColumnList)

Index(['adjustPoints', 'autoCellPoints', 'autoCellsBottom', 'autoCellsInner',
       'autoCellsOuter', 'autoInitLinePoints', 'autoPoints',
       'controlPanelPoints', 'endgamePoints', 'endgameRobot1', 'endgameRobot2',
       'endgameRobot3', 'endgameRungIsLevel', 'foulCount', 'foulPoints',
       'initLineRobot1', 'initLineRobot2', 'initLineRobot3', 'rp',
       'shieldEnergizedRankingPoint', 'shieldOperationalRankingPoint',
       'stage1Activated', 'stage2Activated', 'stage3Activated',
       'stage3TargetColor', 'tba_numRobotsHanging',
       'tba_shieldEnergizedRankingPointFromFoul', 'techFoulCount',
       'teleopCellPoints', 'teleopCellsBottom', 'teleopCellsInner',
       'teleopCellsOuter', 'teleopPoints', 'totalPoints', 'alliance1',
       'alliance2', 'alliance3', 'matchKey'],
      dtype='object')


Now lets make the qualitative data: "TRUE/FALSE" into quantitative "1/0"

In [242]:
sdf.initLineRobot1 = sdf.initLineRobot1.replace({'Exited': 1,'None': 0})
sdf.initLineRobot2 = sdf.initLineRobot2.replace({'Exited': 1,'None': 0})
sdf.initLineRobot3 = sdf.initLineRobot3.replace({'Exited': 1,'None': 0})
sdf.endgameRobot1 = sdf.endgameRobot1.replace({'Hang': 2,'Park': 1,'None': 0})
sdf.endgameRobot2 = sdf.endgameRobot2.replace({'Hang': 2,'Park': 1,'None': 0})
sdf.endgameRobot3 = sdf.endgameRobot3.replace({'Hang': 2,'Park': 1,'None': 0})
sdf.endgameRungIsLevel = sdf.endgameRungIsLevel.replace({'IsLevel': 1,'NotLevel':0})
sdf.shieldEnergizedRankingPoint = sdf.shieldEnergizedRankingPoint.replace({True: 1,False: 0})
sdf.shieldOperationalRankingPoint = sdf.shieldOperationalRankingPoint.replace({True: 1,False: 0})
sdf.tba_shieldEnergizedRankingPointFromFoul = sdf.tba_shieldEnergizedRankingPointFromFoul.replace({True: 1,False: 0})
sdf.stage1Activated = sdf.stage1Activated.replace({True: 1,False: 0})
sdf.stage2Activated = sdf.stage2Activated.replace({True: 1,False: 0})
sdf.stage3Activated = sdf.stage3Activated.replace({True: 1,False: 0})
#useless info on stage3TargetColor, dropping

Sorting by match order:

In [243]:
msdf = sdf
mList = msdf.matchKey
qList=[]
qfList=[]
sfList=[]
fList=[]
NaNList = []

for match in mList:
    if 'qm' in match:
        qList.append(match)
    elif 'qf' in match:
        qfList.append(match)
    elif 'sf' in match:
        sfList.append(match)
    elif 'f1' in match:
        fList.append(match)
    else:
        NaNList.append(match)

eventStorageqm = qList[0].index('qm') + 2
eventName = qList[0][:eventStorageqm]
for i in range(len(qList)):
    qList[i] = int(qList[i][eventStorageqm:])
qList.sort()
for i in range(len(qList)):
    qList[i] = eventName + str(qList[i])
sorter = qList + qfList + sfList + fList + NaNList

In [244]:
msdf = msdf.set_index('matchKey')
msdf = msdf.loc[sorter]

In [245]:
msdf = msdf.drop_duplicates(keep = 'first')

In [246]:
msdf = msdf.drop('stage3TargetColor',1)

C:\Users\cttpa\AppData\Local\Temp/ipykernel_14508/2290944533.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  msdf = msdf.drop('stage3TargetColor',1)


In [247]:
msdf.to_csv('mndu2020MSDF.csv')
msdf.to_pickle('mndu2020MSDF.pkl')

In [248]:
teamList = list(set(msdf['alliance1'].tolist()+msdf['alliance2'].tolist()+msdf['alliance3'].tolist()))
teamList.sort(key=int)
matchMatrix = []
#CHANGED TO ONLY DO QUALS FOR THIS DULUTH, WOULD NEED CHANGING FOR MODULARITY
for i in range(len(qList)):
    a1 = msdf.alliance1[i]
    a2 = msdf.alliance2[i]
    a3 = msdf.alliance3[i]
    matchTeams = []
    for j in range(len(teamList)):
        team = teamList[j]
        if (team == a1) or (team == a2) or (team == a3):
            matchTeams.append(1)
        else:
            matchTeams.append(0)
    matchMatrix.append(matchTeams)
matchNPArray = np.array(matchMatrix)
matchNPArray

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [249]:
scoreList = []
for i in range(len(qList)):
    scoreList.append(msdf.totalPoints[i])
scoreArray = np.array(scoreList).T
scoreArray

array([115,  47,  51, 123,  71,  98,  61, 134,  94,  70,  97, 142,  71,
       104,  21, 108, 107,  35,  91, 111,  96,  54, 122,  64, 123,  58,
       135,  54,  89,  85,  83,  64,  97,  72,  99,  87, 144, 112,  14,
        91, 114,  65, 102,  67, 177,  82,  57, 103, 136,  38, 127, 109,
       122,  58, 118, 102,  83,  38,  53, 146,  52,  90,  82,  58,  99,
       130,  62, 113, 136, 153, 106,  69, 153,  94,  79, 129, 102,  42,
        82,  78,  78,  96,  72,  88, 114, 147,  90,  71,  82,  72,  71,
       116,  93,  80, 143,  74, 116, 127,  57,  63,  57,  76, 134, 103,
        99, 107, 181,  74, 121, 114, 130, 119, 123,  72,  84, 117, 132,
       125,  81, 134,  62, 123, 113, 125, 101,  84,  66, 143, 155, 101,
        56,  64, 144, 124,  81,  76,  56, 113,  51,  47,  89, 146,  70,
       121,  69, 145, 123, 112, 137, 123,  96, 145,  79,  89,  68,  98,
       121, 100,  64,  72, 141, 104,  99, 101,  31, 122, 146,  89, 105,
        66, 116, 107,  83, 103,  74, 108, 127, 104, 123, 140, 10

In [250]:
#create telepoScoreArray
tScoreList = []
for i in range(len(qList)):
    tScoreList.append(msdf.teleopPoints[i])
tScoreArray = np.array(tScoreList).T
tScoreArray

array([ 41,  18,  20,  88,  52,  64,  36,  69,  69,  48,  46, 100,  30,
        33,   6,  66,  54,  30,  75,  89,  46,  23,  77,  27,  51,  29,
        96,  34,  61,  40,  61,  34,  69,  47,  74,  51,  92,  59,   5,
        77,  71,  32,  65,  22, 112,  37,  12,  62,  68,  18,  89,  67,
        90,  50,  60,  64,  48,  13,  20, 111,  29,  32,  39,  16,  78,
       111,  57,  73, 101, 113,  73,  46, 108,  60,  53,  74,  74,  17,
        37,  36,  29,  61,  37,  69,  80, 112,  48,  20,  46,  39,  52,
        41,  62,  20,  95,  36,  84,  87,  47,  24,  25,  43,  89,  59,
        81,  76, 109,  36,  83,  73,  85,  90,  95,  38,  45,  80,  91,
        95,  37,  91,  27,  92,  87,  88,  78,  47,  42, 105,  69,  33,
        44,  37,  89,  64,  54,  32,  23,  78,  28,  32,  52, 100,  37,
        56,  30,  85,  70,  81, 114, 101,  46,  85,  41,  31,  49,  69,
        90,  51,  30,  57, 110,  81,  78,  50,  21,  75,  85,  79,  83,
        36,  69,  86,  54,  60,  60,  75,  68,  52,  67, 107,  6

In [251]:
topr,residuls,rank,singValues = np.linalg.lstsq(matchNPArray,tScoreArray,rcond=None)
print(topr)

[26.12156188 31.89406541 42.21138155 13.88483978 17.10207162 19.1186756
 27.80183048 17.87965227 14.89964235 15.17051684 28.91613617 27.20833934
  7.19091147 14.69485654 28.58843472 21.10211679 19.44577703 24.51265463
  1.81308583 25.20008421 26.49089407 15.3974187  14.52988972 21.4987642
  2.4389207  10.16545142 -7.39614014 36.55863796  5.46000283 16.24183059
 27.69182646  9.38653801 31.84823396 36.75191426 24.80687111  2.73954775
 11.34064804 30.27783201 24.4751738   6.24799582 30.54001259 12.56772355
 14.4849744  48.17300805  8.45311247 53.41738604  8.70273787 37.5383032
 19.03402396  8.18296936 20.72241438 28.86167313 19.74739849 30.21171546
 32.59984257 19.05798075 26.03026629 24.57704093 27.3484402   9.38647527
  1.20414628  5.71890358 16.99879977]


In [252]:
aScoreList = []
for i in range(len(qList)):
    aScoreList.append(msdf.autoPoints[i])
aScoreArray = np.array(aScoreList).T

In [253]:
aopr,ared, arank,asingValues = np.linalg.lstsq(matchNPArray,aScoreArray,rcond=None)

In [254]:
eScoreList = []
for i in range(len(qList)):
    eScoreList.append(msdf.endgamePoints[i])
eScoreArray = np.array(eScoreList).T

In [255]:
eopr, ered, erank,esingValues = np.linalg.lstsq(matchNPArray,eScoreArray,rcond=None)

In [256]:
opr, residuals, rank, singValues = np.linalg.lstsq(matchNPArray,scoreArray,rcond=None)
print(opr)

[32.14444685 43.7477653  67.68949159 30.50903958 39.45738974 26.65569424
 47.28045366 28.05503933 13.77389213 30.73445816 41.08182077 37.84251524
 21.76665291 28.83599986 44.59628504 38.42014825 36.19624061 43.77430938
 13.39822124 40.70840347 39.1735908  21.78038236 26.50262427 45.56416519
  1.59942857 14.34943308 -1.4949135  41.03270866  3.93304713 35.82392936
 32.93954428 22.9734413  48.93419183 47.65563229 32.59678376 21.65287056
 16.94450293 48.00731499 39.34831433 20.13740992 44.7331575  25.91212439
 23.7255723  60.4890345  26.6008905  56.387855   19.75178221 52.70604037
 42.34730585  4.57921508 37.50195334 47.80197686 41.68911639 33.30952499
 42.22240834 32.62273257 36.13628997 34.17646943 46.70389735 18.99102869
  8.40744238 12.58707803 32.53041901]


In [257]:
#dpr
dScoreList = []
for i in range(0,len(qList),2):
    dScoreList.append(msdf.totalPoints[i+1])
    dScoreList.append(msdf.totalPoints[i])
dScoreArray = np.array(dScoreList).T

In [258]:
dopr,dResid,dRank,dSing = np.linalg.lstsq(matchNPArray,dScoreArray,rcond=None)
print(dopr)

[37.72771728 23.41504518 37.4577414  41.05437751 14.87265832 55.28175902
 35.08299035 20.54886168 38.69272436 22.10084214 26.21041989 28.1775664
 52.18924667 28.17647142 15.04308387 31.08370952 12.96529084 43.49141258
 24.17604215 38.63158857 18.04642808 46.51039525 42.8734776  40.25934246
 27.71239751 24.45316651 26.596906   40.79036534 34.65808844 48.91407617
 19.12434302 23.37417623 21.33521012 41.67623816 14.71033092 40.11788814
 53.30259997 34.17456034 37.95251973 27.75165145 28.87273924 56.45709981
 29.88373555 46.42485004 37.65009918 30.52144772 36.51346805 11.79653712
 18.87071719 22.53724045 35.08267224 43.57321202 29.68293835 31.09838836
 39.97003718 42.59625038 31.63779696 16.76789601 37.01585292 12.62723069
 34.00537258 35.65735803 46.64226992]


In [259]:
ccwm = []
for i in range(len(opr)):
    ccwm.append(opr[i]-dopr[i])

In [260]:
rankList=[]
for team in teamList:
    rank = dict(tba.team_status('frc'+team,event))['qual']['ranking']['rank']
    rankList.append(rank)
print(rankList)

[37, 7, 1, 54, 17, 47, 31, 42, 43, 20, 8, 46, 56, 21, 26, 33, 6, 28, 39, 52, 12, 62, 45, 19, 60, 48, 63, 34, 30, 59, 15, 11, 14, 5, 38, 51, 61, 3, 40, 55, 25, 49, 41, 22, 35, 4, 53, 2, 9, 44, 29, 24, 27, 36, 23, 57, 13, 16, 10, 32, 58, 50, 18]


In [261]:
teamVar = pd.DataFrame({'Rank':rankList,'OPR':opr.tolist(),'DPR':dopr.tolist(),'CCWM':ccwm,'teleopOPR':topr.tolist(),'autoOPR':aopr.tolist(),'endgameOPR':eopr.tolist()},index=teamList)
teamVar.to_csv('2020mnduQUALSOPR.csv')

Elo Implementation:
P1 = Probability of winning of alliance with rating2
P2 = Probability of winning of alliance with rating1

In [262]:
#team = msdf.alliance1.loc['2020mndu2_qm1'][0]
#teamVar.OPR.loc[team]

In [263]:
teamVar
#We are curious about how we can use OPR to rate event strength, lets see how the matches stack up based on OPR
#We have the match matrix now or we can use match schedule dataframe, dataframes are nicer :)
#qList contains all match keys of qual matches, which is what we are interested in using to determine event difficulty for ALL robots, not just those who made it to the playoffs, as those matches will obviously be strong. Lets see how the average OPR stacks up for each alliance
muBlueOPRlist = []
muRedOPRlist = []
sigmaBlueOPRlist = []
sigmaRedOPRlist = []
b1List = []
b2List= []
b3List = []
r1List = []
r2List = []
r3List = []
winList = []
for i in range(0,len(qList),2):
    #Get alliance strings
    b1 = msdf.alliance1.loc[qList[i]][0]
    b2 = msdf.alliance2.loc[qList[i]][0]
    b3 = msdf.alliance3.loc[qList[i]][0]
    b1OPR = teamVar.OPR.loc[b1]
    b2OPR = teamVar.OPR.loc[b2]
    b3OPR = teamVar.OPR.loc[b3]

    muBlueOPR = (b1OPR+b2OPR+b3OPR)/3
    sigmaBlueOPR = stat.stdev([b1OPR,b2OPR,b3OPR])

    muBlueOPRlist.append(muBlueOPR)
    sigmaBlueOPRlist.append(sigmaBlueOPR)

    b1List.append(b1)
    b2List.append(b2)
    b3List.append(b3)

    r1 = msdf.alliance1.loc[qList[i]][1]
    r2 = msdf.alliance2.loc[qList[i]][1]
    r3 = msdf.alliance3.loc[qList[i]][1]
    r1OPR = teamVar.OPR.loc[r1]
    r2OPR = teamVar.OPR.loc[r2]
    r3OPR = teamVar.OPR.loc[r3]

    muRedOPR = (r1OPR+r2OPR+r3OPR)/3
    sigmaRedOPR = stat.stdev([r1OPR,r2OPR,r3OPR])

    muRedOPRlist.append(muRedOPR)
    sigmaRedOPRlist.append(sigmaRedOPR)

    r1List.append(r1)
    r2List.append(r2)
    r3List.append(r3)

    if(msdf.totalPoints[i] > msdf.totalPoints[i+1]):
        winList.append('Blue')
    else:
        winList.append('Red')
#Match OPR DataFrame
from collections import OrderedDict #for removing duplicates in qList as we have one match with both red and blue values
modf = pd.DataFrame(data = {'r1':r1List,'r2':r2List,'r3':r3List,'b1':b1List,'b2':b2List,'b3':b3List,'muRedOPR':muRedOPRlist,'muBlueOPR':muBlueOPRlist,'sigmaRedOPR':sigmaRedOPRlist,'sigmaBlueOPR':sigmaBlueOPRlist,'win':winList},index=list(OrderedDict.fromkeys(qList)))
modf

,r1,r2,r3,b1,b2,b3,muRedOPR,muBlueOPR,sigmaRedOPR,sigmaBlueOPR,win
2020mndu_qm1,7068,4217,3275,1816,6045,3102,25.485288,46.290932,23.853262,19.724670,Blue
2020mndu_qm2,4728,7797,4511,2861,2177,2823,42.859211,32.823991,8.662759,7.436591,Red
2020mndu_qm3,4009,2264,7864,3755,4166,5826,37.278549,18.234314,15.860689,14.406318,Red
2020mndu_qm4,6146,4674,6628,8416,2169,3754,31.032709,28.273280,7.393615,5.713124,Red
2020mndu_qm5,5991,3291,5464,4238,7041,6022,36.378130,30.925126,21.648179,24.800572,Blue
...,...,...,...,...,...,...,...,...,...,...,...
2020mndu_qm91,2175,5991,4741,5542,3058,5253,47.833626,26.544265,8.466636,9.813867,Red
2020mndu_qm92,3755,4238,3122,5576,7041,3267,25.010652,39.021896,20.372634,4.946188,Blue
2020mndu_qm93,3291,5690,4480,2526,6146,3740,36.570395,30.149812,23.554279,14.206571,Blue
2020mndu_qm94,93,2823,1816,6453,3100,3754,43.522799,28.589815,20.940840,11.806175,Red


In [264]:
modf.to_csv('mnduMatchOPRs.csv')

In [265]:
muDiffList = []
for (rV, bV) in zip(muRedOPRlist,muBlueOPRlist):
    muDiffList.append(rV-bV)
muDiffList

[-20.805643996949744,
 10.03522009622894,
 19.044234733752592,
 2.7594287528351273,
 5.453004369096302,
 3.313936228779717,
 6.000026944228573,
 7.942733776668607,
 -14.713979235731077,
 2.9681943124943473,
 -13.382131352739279,
 -20.127499337157698,
 -7.613083864648349,
 -20.33463923056582,
 0.9536095571005028,
 0.06253210571843582,
 8.545216662031244,
 -0.9886032217371152,
 -13.40735658683451,
 5.629167233464113,
 -15.787414996078294,
 0.5950963461278569,
 -18.198874996033233,
 18.925584252726615,
 -6.826331949982453,
 -1.077987332690597,
 -17.317944315579762,
 2.0508476401345206,
 -17.197483314002096,
 20.652678860219552,
 19.601907038800956,
 -5.361280303899168,
 9.561910767845305,
 15.075462609359302,
 2.9727133180772114,
 0.24792347949004068,
 -10.880930084149078,
 6.525295556151654,
 -8.601832280188603,
 -10.160993742182768,
 -7.181138825776635,
 -4.870102837813878,
 15.768652625938614,
 -2.0347345732764133,
 -0.09667909825542864,
 6.837029232214064,
 1.1552189406819693,
 -3.821

In [266]:

#Average OPR Diff for Schedule,  Opponent's Average OPR - Team's Average OPR, 0 indicates Average schedule, negative indicates easy schedule
avgOPR = teamVar.OPR.mean()
scheduleStrength = {}
for team in teamList:
    scoreList = []
    scoreAvg = 0
    for i in range(1,len(qList),2):
        matchNumber = int((i/2)+0.5)
        b1 = msdf.alliance1.loc[qList[i]][0]
        b2 = msdf.alliance2.loc[qList[i]][0]
        b3 = msdf.alliance3.loc[qList[i]][0]
        r1 = msdf.alliance1.loc[qList[i]][1]
        r2 = msdf.alliance2.loc[qList[i]][1]
        r3 = msdf.alliance3.loc[qList[i]][1]
        if team in [b1,b2,b3]:
        #Team is on blue alliance, grab red alliance average OPR
            scoreList.append(modf.muRedOPR.loc[event+'_qm'+str(matchNumber)] - (((modf.muBlueOPR.loc[event+'_qm'+str(matchNumber)]*3) - teamVar.OPR.loc[team] + avgOPR )/3))
        elif team in [r1,r2,r3]:
            scoreList.append(modf.muBlueOPR.loc[event+'_qm'+str(matchNumber)] - (((modf.muRedOPR.loc[event+'_qm'+str(matchNumber)]*3) - teamVar.OPR.loc[team] + avgOPR )/3))
    scheduleStrength[team] = stat.mean(scoreList)


In [267]:
ssdf=pd.DataFrame.from_dict(data=scheduleStrength,orient='index',columns=['scheduleStrength'])
ssdf.to_csv("scheduleStrength.csv")


In [268]:
teamVar['scheduleStrength'] = ssdf.scheduleStrength

In [269]:
teamVar

,Rank,OPR,DPR,CCWM,teleopOPR,autoOPR,endgameOPR,scheduleStrength
93,37,32.144447,37.727717,-5.583270,26.121562,5.119497,19.175494,3.535110
167,7,43.747765,23.415045,20.332720,31.894065,13.209221,20.984938,-2.982333
1816,1,67.689492,37.457741,30.231750,42.211382,18.732440,27.846314,-0.894715
2169,54,30.509040,41.054378,-10.545338,13.884840,13.404815,0.311722,-2.288619
2175,17,39.457390,14.872658,24.584731,17.102072,9.361796,7.582447,-1.931328
...,...,...,...,...,...,...,...,...
7797,10,46.703897,37.015853,9.688044,27.348440,13.317707,20.437767,2.621150
7864,32,18.991029,12.627231,6.363798,9.386475,9.943455,7.128608,-5.800205
7893,58,8.407442,34.005373,-25.597930,1.204146,8.665963,-0.893819,2.389284
8372,50,12.587078,35.657358,-23.070280,5.718904,1.874498,6.180580,1.518821


In [270]:
teamVar.loc['']
teamVar.to_csv('teamVariables.csv')

KeyError: ''

In [ ]:
def visualize(x,y):
    plt.scatter(x,y)
    theta = np.polyfit(x,y,1)
    y_line = theta[1] + theta[0] * x
    plt.plot(x,y_line,'r')
    plt.xlabel(x.name)
    plt.ylabel(y.name)
    plt.show()
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    print("Slope: "+ str("%.2f"%slope) + " Intercept: " + str("%.2f" %intercept) + " R Value: " + str("%.2f" %r_value) + " R^2: "+str("%.2f" %r_value**2) + " P Value: "+str("%.2f" %p_value)+" Standard Error: " + str("%.2f" %std_err))

In [ ]:
visualize(teamVar.OPR,teamVar.Rank)

In [ ]:
dict(tba.event_predictions(event))
